In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [8]:
engine = create_engine("sqlite:///HawaiiDatasqlite")

In [9]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [10]:
# We can view all of the classes that automap found
Base.classes.keys()

[]

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Look at data column names and types
inspector = inspect(engine)
columns = inspector.get_columns('Measurement')
for c in columns:
    print(c['name'], c["type"])
    
print(f"-------------------------------")

inspector = inspect(engine)
columns = inspector.get_columns('Station')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results


# Find the last data point
last_data = session.query(func.max(Measurement.date)).scalar()
print(last_data)
print(type(last_data))

# Calculate the date 1 year ago from the last data point in the database
year =  dt.timedelta(days=365)
year_prior = dt.datetime.strptime(last_data,'%Y-%m-%d') - year
print(year)
print(year_prior)

# Perform a query to retrieve the date and precipitation scores
precip_date = session.query(Measurement.date).filter(Measurement.date > year_prior).all()
precip = session.query(Measurement.prcp).filter(Measurement.date > year_prior).all()

# List comprehension
precip_date_list = list(np.ravel(precip_date))
print(len(precip_date_list))
print(precip_date_list)


precip_list = list(np.ravel(precip))
print(len(precip_list))
print(precip_list)


# Save the query results as a Pandas DataFrame and set the index to the date column
precip_data = {"Date": precip_date_list, 'Precipitation (inches)': precip_list}
precip_df = pd.DataFrame(precip_data, columns=['Date', 'Precipitation (inches)'])
precip_df.set_index('Date', inplace=True)
print(precip_df)


# Sort the dataframe by date
precip_df.sort_values('Date')
print(precip_df)




In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
precip_df.plot(x_compat=True)
plt.title('One Year of Precipitation in Hawaii')
loc = np.arange(len(precip_df))
plt.xticks(loc)
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precip_df.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?
station = session.query(Station).count()
print(station)

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?

station_active = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()


# List the stations and the counts in descending order.
print(station_active)


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
most_active_station = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).filter(Measurement.station == 'USC00519281').all() 
print(most_active_station)


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station 

# Perform a query to retrieve the precipitation observations for the most active station
most_active_station_data = session.query(Measurement.tobs).filter(Measurement.date > year_prior).all()

# List comprehension
most_active_station_data_list = list(np.ravel(most_active_station_data))

# Plot the results as a histogram 
most_active_station_data_hist = plt.hist(most_active_station_data_list, bins=12, label='Temperature Observation')

# label the x axis
plt.xlabel('Temperature (F)')

#label the y axis
plt.ylabel('Frequency')

#set the title
plt.title(f'One Year of Temperature Observation Data from Hawaiis Most Active Station: USC00519281')

plt.legend()
plt.show()

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
# Dates of my trip
start_date = '2019-06-18'
end_date = '2019-07-02'

# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
avg_temp_trip = session.query(Measurement.tobs).filter(Measurement.date.calc_temps(start_date, end_date)).all() 
print(avg_temp_trip)


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

